In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import networkx.algorithms.community as nx_comm
from sklearn.preprocessing import StandardScaler
from pingouin import partial_corr

import neurocognitive_analysis_utils as cog

import warnings
warnings.filterwarnings('ignore')

In [ ]:
infile_clusters  = 'results/clusters_genetics.csv'
infile_adnimerge = 'data/ADNI/ADNIMERGE_processed.csv'
infile_metadata = "data/ADNI/neuropsychological/ADAS_Metadata.csv"

clusters  = pd.read_csv(infile_clusters, index_col='PTID')
clusters['cluster'].replace({0:1, 1:2, 2:3}, inplace=True)

adnimerge = pd.read_csv(infile_adnimerge, index_col='PTID')
adnimerge['DX_bl'].replace({'AD':'Dementia', 'EMCI':'MCI', 'LMCI':'MCI', 'SMC':'CN'}, inplace=True)

results = pd.concat([adnimerge['RID'], clusters[['DX', 'DX_bl', 'cluster']]], axis=1, join='inner')
results['DX_bl'].replace({'AD':'Dementia', 'EMCI':'MCI', 'LMCI':'MCI', 'SMC':'CN'}, inplace=True)
results = results.loc[(results['DX_bl'] == 'Dementia') | (results['DX_bl'] == 'MCI')]
results = results.set_index('RID')

In [ ]:
adas_GO23 = pd.read_csv('data/ADNI/neuropsychological/ADAS_ADNIGO23.csv', index_col='RID')
adas_GO23 = adas_GO23[adas_GO23['VISCODE2']=='bl']
print(adas_GO23.shape[0])

adas_columns = ["Q1SCORE","Q2SCORE","Q3SCORE",
                "Q4SCORE","Q5SCORE","Q6SCORE",
                "Q7SCORE","Q8SCORE","Q9SCORE",
                "Q10SCORE","Q11SCORE","Q12SCORE",
                "Q13SCORE"]

#Drop NaN
adas_GO23 = adas_GO23.dropna(subset=adas_columns)
print("Number of subjects that have completed all tests: ", adas_GO23.shape[0])

#group variables by cognitive test
adas_metadata = pd.read_csv(infile_metadata, sep=";")
adas_GO23 = cog.group_by_test(adas_metadata, adas_GO23)

Data scaling using CN subjects

In [ ]:
#get test results for the controls

adas_columns = adas_metadata['Label'].unique()

adas_CN = adas_GO23[adas_GO23.index.isin(adnimerge[adnimerge["DX_bl"]=='CN'].reset_index()['RID'])]
adas_CN = adas_CN[adas_columns]

adas_DX = pd.concat([adas_GO23, results], axis=1, join='inner')
adas_DX = adas_DX[adas_columns]
# # #filter table to show only tests results
# # # adas_columns = adas_metadata['Label'].unique()

print("Number of controls: ", adas_CN.shape[0])
print("Number of MCI/Dem: ", adas_DX.shape[0])

scaler = StandardScaler()
scaler = scaler.fit(adas_CN)

x_adas = scaler.transform(adas_DX) # scale data
x_adas = pd.DataFrame(x_adas, columns=adas_columns, index=adas_DX.index)
print("Number of MCI/Dem: ", x_adas.shape[0])
print('data scaled!')


In [ ]:
sns.set_style('white')
palette = {'Attention':"#9999FF", 'Executive':"#CC99FF", 'Language':"#F8766D",
           'Memory':"#00BFC4", 'Orientation':"#66CC99", 'Visuospatial':"#FFCC99"}
order = ['Attention', 'Executive', 'Language', 'Memory', 'Orientation', 'Visuospatial']
print()

In [ ]:
for variable in ['cluster']:

    print("----------------------")
    print(variable)
    print("----------------------")
    
    # dataframe to store global metrics for each cluster
    gm_columns = ['Cluster','NNodes', 'NEdges', 'Diameter', 'Density', 'AvDegree', 'Transitivity','AvCC']
    gm_df = pd.DataFrame(columns=gm_columns) 

    # dataframe to store centrality metrics for each cluster
    node_ls = list(x_adas.columns)
    cm_columns = ['cluster', 'Metric'] + node_ls
    dc_df = pd.DataFrame(columns=cm_columns) # degree centrality
    bc_df = pd.DataFrame(columns=cm_columns) # betweeness centrality
    
    for cluster in [1, 2, 3]: 
        
        tmp_cluster = results.loc[results['cluster'] == cluster]
        ids_cluster = tmp_cluster.index
        print("Number of patients in cluster", cluster, ": ", len(ids_cluster))
        
        adas_cluster = pd.concat([x_adas, tmp_cluster], axis=1, join='inner')
        adas_cluster = adas_cluster[x_adas.columns]
        print(f"Number of patients with ADAS: {adas_cluster.shape[0]}")

        x = adas_cluster
        
        matrices = cog.bootstrap(x, n_size=10, n_rep=250)
        mx = cog.adj_matrix(matrices, x.columns)

        graph = cog.cognitive_network(mx) #convert corr coefficients into absolute values
        
        infile_metadata = "data/ADNI/neuropsychological/ADAS_Metadata.csv"
        cog.node_attributes(infile_metadata, graph, x)

        labels = {}
        for i in range(mx.shape[1]):
            labels[i] = mx.columns[i]

        mapping = {'Attention':0, 'Executive':1, 'Language':2, 'Memory':3, 'Orientation':4, 'Visuospatial':5}

        if cluster == 1:
            posg = nx.spring_layout(graph, weight='weight', seed=0)
        else:
            posg = posg
            
        cog.plot_cognitive_network(graph, posg, f'Cluster {cluster}')
        
        # save graph as pickle file
        graph_path = f"results/cognitive_profiles/ADAS_graph_cluster{cluster}.gpickle"
        nx.write_gpickle(graph, graph_path)
        
        # graph metrics
        gm_df, dc_df, bc_df = cog.bootstrap_metrics(node_ls, matrices, cluster, gm_df, dc_df, bc_df)
        
        print()
    

    dc_df['cluster'].replace({1:'Cluster 1', 2:'Cluster 2', 3:'Cluster 3'}, inplace=True)
    bc_df['cluster'].replace({1:'Cluster 1', 2:'Cluster 2', 3:'Cluster 3'}, inplace=True)

    # export metrics
    # dc_df.to_csv('results/cognitive_profiles/degree_centrality.csv', index=False)
    # bc_df.to_csv('results/cognitive_profiles/betweeness_centrality.csv', index=False)

In [ ]:
sns.set(style="whitegrid", font='Arial', font_scale=1.)
meanprops = {'markeredgecolor': 'black',  'markerfacecolor': 'white', 'markersize': 8}

In [ ]:
import utils_description as utils

gm_df['Cluster'].replace({1:0, 2:1, 3:2}, inplace=True)

results_global = utils.stats_numerical(gm_df, 'Cluster', round_mean=2, round_std=2)
print(results_global[['cluster1', 'cluster2', 'cluster3', 'F', 'pvalue', 'significant', 'tukey_HSD']].to_latex())

In [ ]:
# centrality measures plot
palette = sns.hls_palette(n_colors=3)

fig, axs = plt.subplots(1, 3, figsize=(10, 4))

df = pd.melt(dc_df, id_vars=['cluster', 'Metric'], var_name='Node', value_name='Value')

c1 = df.loc[df['cluster'] == 'Cluster 1']
c2 = df.loc[df['cluster'] == 'Cluster 2']
c3 = df.loc[df['cluster'] == 'Cluster 3']


o1 = c1.groupby(by=['Node'])['Value'].mean().sort_values(ascending=False).index
o2 = c2.groupby(by=['Node'])['Value'].mean().sort_values(ascending=False).index
o3 = c3.groupby(by=['Node'])['Value'].mean().sort_values(ascending=False).index

c1 = c1.set_index('Node').loc[o1].reset_index()
c2 = c2.set_index('Node').loc[o2].reset_index()
c3 = c3.set_index('Node').loc[o3].reset_index()

sns.barplot(data=c1, x='Value', y='Node', color=palette[0], errorbar='sd', ax=axs[0])
sns.barplot(data=c2, x='Value', y='Node', color=palette[1], errorbar='sd', ax=axs[1])
sns.barplot(data=c3, x='Value', y='Node', color=palette[2], errorbar='sd', ax=axs[2])

axs[0].set_xlim(0, 10)
axs[1].set_xlim(0, 10)
axs[2].set_xlim(0, 10)

axs[0].set_xlabel('')
axs[1].set_xlabel('')
axs[2].set_xlabel('')

axs[0].set_title('Cluster 1')
axs[1].set_title('Cluster 2')
axs[2].set_title('Cluster 3')

plt.tight_layout()
plt.savefig('figures/local_metrics_cognitive.pdf', dpi=300)